In [2]:
# %load write_mat.py
##
#   load .mat files and write the contens to a 
#   .mat file.
#    the files has two fields - 'inputs' and 'targets'  
#   'inputs' is a numpy array of tuples
#   'targets' is from 0 to 7 (for grid 'A' to 'H')

import scipy.io
import numpy as np
import os.path
import re

inputs = []
targets = []

def init():
    inputs = []
    targets = []

def loadMat(file):
    '''
     load matfile named 'file' ..
    '''
    return scipy.io.loadmat(file)

def load_local_features(data,target_value):
    ''' load local features from data '''
    feature_name = []

    # extract the local feature name
    for i in data:
        if not re.match(r'__*',i) and i!='F' :
            if not re.match(r'global*',i): # skipping global feature
                if not re.match(r'wav_var',i) : 
                #if re.match(r'mean',i) or re.match(r'diff',i) or re.match(r'range',i):
                    feature_name.append(i)
    input  = zip( *[ data[i][0] for i in feature_name ])

    target =  [[target_value] for i in xrange(len(input))] 

    inputs.extend(input)
    targets.extend(target)

def write_to_file(filename):
    '''
        write inputs and targets to a filename .mat file
     '''
    #np.savez(filename,inputs,targets)
    scipy.io.savemat(filename, mdict = {'inputs':inputs,'targets':targets} )



def main():
    grids = ['A','B','C','D','E','F','G','H'];
    init()

    for grid in grids:
        #filename = '../feature_data/matfile/{0}/{0}_P{1}.mat'\
        #.format(grid,i)
        filename = '../mat/me/features/Grid{0}.mat'\
            .format(grid)
        if os.path.exists(filename):
            load_local_features(
                loadMat(filename), ord(grid)-ord('A')
            )
    file_to_save = 'feature_data/features.mat'
    write_to_file(file_to_save)

if __name__ == '__main__':
    main()






 

In [3]:
main()

In [8]:
# %load svm_1.py
from __future__ import division
import scipy.io
import numpy as np
import scipy as sp
import sys
import random
from sklearn import svm

allf = scipy.io.loadmat('feature_data/features.mat')
inputs = allf['inputs']
targets = allf['targets']

assert len(inputs)==len(targets)
percent = 0.75 # default 75% train data

#if len(sys.argv)>1:
#    percent = float(sys.argv[1])

z = zip(inputs,targets)
random.shuffle(z)

train_last_index = int( percent * len(z) )

# train inputs
X = np.array([ z[i][0] for i in xrange(train_last_index) ])
Y = np.array([ int(z[i][1]) for i in xrange(train_last_index) ])

assert len(X)==len(Y)

clf = svm.SVC()
clf.fit(X,Y)

test_x = np.array([ z[i][0] for i in xrange( train_last_index,len(z) )  ])
test_y = np.array([ int(z[i][1]) for i in xrange(train_last_index,len(z)) ])

pr = clf.predict(test_x)

d = abs(pr-test_y)

s = sum([1  for i in d if i!=0])

print 'Error percent :',s/len(d)*100



Error percent : 23.1366459627


In [18]:
%run svm_1.py

Error percent : 21.5838509317


In [19]:
allf['inputs'][0]

array([ -6.24545679,  -0.7109509 ,  -5.4403286 ,  60.02294948,
        60.02294948, -13.88056216])

In [20]:
%run write_mat.py

In [26]:
%run svm_1.py

Error percent : 28.2608695652


In [24]:
allf['inputs']

array([[ -6.24545679,  -0.7109509 ,  -5.4403286 , ...,  60.02294948,
        -13.88056216, -62.35149755],
       [ -6.36115672,  -0.57511224,  -5.60568865, ...,  60.02297139,
        -13.92614652, -61.54056734],
       [ -6.50407888,  -0.63179481,  -5.92923723, ...,  60.02243203,
        -14.52637645, -62.35149755],
       ..., 
       [ -4.53467648,  -1.65715016,  -2.21450199, ...,  50.04282168,
         -6.42760601, -65.12408627],
       [ -4.51716009,  -1.73587005,  -2.80303806, ...,  49.98749974,
         -8.04276505, -63.73779191],
       [ -4.94758398,  -1.38298702,  -3.09736442, ...,  49.92780845,
         -8.69964734, -62.35149755]])

In [27]:
allf['inputs'][0]

array([ -6.24545679e+00,  -7.10950898e-01,  -5.44032860e+00,
         6.00229495e+01,   6.00229495e+01,  -1.38805622e+01,
         8.33851424e-28])

In [30]:
%run svm_1.py

Error percent : 34.4720496894


In [31]:
allf['inputs'][0]

array([ -3.38813179e-21,   2.83540091e-03,  -7.10950898e-01,
        -5.44032860e+00,   6.00229495e+01,  -1.38805622e+01,
         3.38813179e-21,   0.00000000e+00,   6.77626358e-21])

In [34]:
re.match(r'd\d','d1')

In [35]:
%run write_mat.py

In [36]:
%run svm_1.py

Error percent : 24.8447204969


In [37]:
allf['inputs'][0]

array([ -6.24545679,  -0.7109509 ,  -5.4403286 ,  60.02294948, -13.88056216])

In [5]:
# %load load.py
from __future__ import  division
import numpy as np
import scipy as sp
%matplotlib inline
from matplotlib import pyplot as plt
import scipy.io
import sklearn
from sklearn import svm

In [6]:
filename = 'feature_data/features.mat'

data = scipy.io.loadmat(filename)


In [9]:
len(data['inputs'])

1287

In [19]:
len(data['inputs'][0])

12

In [73]:
%%writefile normalize_data.py
import numpy as np
import scipy as sp
import scipy.io


filename = 'feature_data/features.mat'

data = scipy.io.loadmat(filename)
inputs = data['inputs']
feature_number = len(inputs[0]) # number of features per vector

norma_val = 10 # normalize value

max_value = np.zeros(feature_number) # maximum value of ith feature
f_value = np.zeros((feature_number,len(inputs) ))

#n_inputs = np.zeros((len(inputs,feature_number))) # normalize inputs

## find maximum values of each feature
for i in xrange(feature_number):
    max_value[i] = np.max( [ abs(k[i]) for k in inputs ] )
    weight = norma_val/max_value[i] # normalize between +- 10
    f_value[i] = np.array([ k[i]*weight for k in inputs ])
    
n_inputs = zip( *f_value[:] )
scipy.io.savemat('feature_data/normal_features.mat',mdict={'inputs':n_inputs,'targets':data['targets']})

Overwriting normalize_data.py


In [74]:
%run normalize_data.py

In [67]:
n_inputs[:2]

[(2.308580718377684e-05,
  -7.642970757540736,
  0.58694520920138893,
  -3.7021404006874725,
  -7.421810782576487,
  9.9997853621218802,
  9.999785362121882,
  -7.9153431200420208,
  2.9662562577317188e-05,
  9.2463373018116909e-05,
  0.0002685655344794162,
  3.2653061224489797),
 (7.7994476020605941e-06,
  -7.7845602745017279,
  0.45976486738977146,
  -2.9947866737450086,
  -7.6473984413556035,
  9.9997890116431005,
  9.9997890116431059,
  -7.9413374447552512,
  5.4394972213150739e-05,
  0.00038694812453202941,
  0.0002195896311967349,
  7.3469387755102051)]

In [68]:
inputs[:2]

array([[  2.44786106e-07,  -6.24545679e+00,   2.83540091e-03,
         -7.10950898e-01,  -5.44032860e+00,   6.00229495e+01,
          6.00229495e+01,  -1.38805622e+01,   9.85148921e-08,
          4.48663767e-08,   2.93325059e-07,   8.33851424e-28],
       [  8.27000065e-08,  -6.36115672e+00,   2.22102115e-03,
         -5.75112244e-01,  -5.60568865e+00,   6.00229714e+01,
          6.00229714e+01,  -1.39261465e+01,   1.80655829e-07,
          1.87760404e-07,   2.39833982e-07,   1.87616570e-27]])

In [69]:
max_value

array([  1.06033159e-01,   8.17150423e+00,   4.83077614e-02,
         1.92037800e+00,   7.33019038e+00,   6.00242378e+01,
         6.00242378e+01,   1.75362735e+01,   3.32118615e-02,
         4.85234047e-03,   1.09219174e-02,   2.55366999e-27])

In [70]:
inputs[0][0]/max_value[0]

2.3085807183776839e-06

In [49]:
a[:][0]=2

In [50]:
a

array([[ 2.,  2.,  2.],
       [ 2.,  3.,  4.]])

In [51]:
a[:][1]=3

In [52]:
a

array([[ 2.,  2.,  2.],
       [ 3.,  3.,  3.]])

In [53]:
a[1][2]

3.0

In [58]:
zip( *a[:] )

[(2.0, 3.0), (2.0, 3.0), (2.0, 3.0)]

In [55]:
inputs

array([[  2.44786106e-07,  -6.24545679e+00,   2.83540091e-03, ...,
          4.48663767e-08,   2.93325059e-07,   8.33851424e-28],
       [  8.27000065e-08,  -6.36115672e+00,   2.22102115e-03, ...,
          1.87760404e-07,   2.39833982e-07,   1.87616570e-27],
       [  9.57166006e-09,  -6.50407888e+00,   1.92392930e-03, ...,
          1.33905844e-08,   1.28066762e-07,   8.33851424e-28],
       ..., 
       [  1.35053314e-03,  -4.53467648e+00,   0.00000000e+00, ...,
          2.16573685e-05,   3.88656197e-05,   5.21157140e-29],
       [  2.07641384e-04,  -4.51716009e+00,   0.00000000e+00, ...,
          3.19815998e-05,   2.39118764e-05,   2.08462856e-28],
       [  6.87735486e-05,  -4.94758398e+00,   4.14913168e-06, ...,
          6.54402088e-06,   2.91970820e-05,   8.33851424e-28]])

In [75]:
dat = scipy.io.loadmat('feature_data/normal_features.mat')

In [79]:
len(dat['inputs'])

1287

In [80]:
dat = scipy.io.loadmat('feature_data/features.mat')

In [81]:
len(dat['inputs'])

1287

In [85]:
%run svm_1.py

Error percent : 36.9565217391


In [86]:
allf['inputs'][:3]

array([[-7.64297076, -3.7021404 , -7.42181078,  9.99978536,  9.99978536,
        -7.91534312,  3.26530612],
       [-7.78456027, -2.99478667, -7.64739844,  9.99978901,  9.99978901,
        -7.94133744,  7.34693878],
       [-7.95946339, -3.28995024, -8.08879023,  9.99969915,  9.99969915,
        -8.28361651,  3.26530612]])